In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install transformers torch trl pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 3.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 8.5 MB/s eta 0:00:00


In [2]:
import random

import numpy as np
import torch
import pandas as pd

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    default_data_collator,
)


def set_seed(seed_val=42):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)



output_dir = "supervised-summarize-checkpoint"
train_batch_size = 16
gradient_accumulation_steps = 1
learning_rate = 1e-5
eval_batch_size = 1
eval_steps = 500
max_input_length = 550
save_steps = 1000
num_train_epochs = 20
random.seed(42)



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
df = pd.read_parquet("/kaggle/input/data-model-files/test_policy.parquet")

In [4]:
df.iloc[12]

prompt    SUBREDDIT: r/tifu\nTITLE: TIFU bY brushing wit...
label     Brush Teeth with Baking Soda without research,...
Name: 12, dtype: object

In [5]:
import json

import pandas as pd
import torch
from datasets import load_dataset
from torch.utils.data import Dataset


class TLDRDataset(Dataset):
    def __init__(self, train_path, tokenizer, split, max_length=256):
        self.post_list = []
        dataset = pd.read_parquet(train_path)
        self.labels = []
#         dataset = dataset[:100]
        for sample in dataset.iterrows():
            self.post_list.append(sample[1]["prompt"])
            self.labels.append(sample[1]["label"])

        self.tokenizer = tokenizer
        self.max_length = max_length
        self.input_ids = []
        self.attn_masks = []

    def __len__(self):
        return len(self.post_list)

    def __getitem__(self, idx):
        txt = self.post_list[idx]
        label = self.labels[idx]

        encodings_dict = self.tokenizer(txt, truncation=True, max_length=self.max_length, padding="max_length")
        encodings_dict_label = self.tokenizer(label,truncation=True, max_length=self.max_length, padding="max_length")
        input_ids = torch.tensor(encodings_dict["input_ids"])
        attn_masks = torch.tensor(encodings_dict["attention_mask"])
        labels_ids = torch.tensor(encodings_dict_label["input_ids"])
        return {
            "input_ids": input_ids,
            "attention_mask": attn_masks,
            "labels": labels_ids,
        }



In [6]:
tokenizer = AutoTokenizer.from_pretrained("bigcode/tiny_starcoder_py")
model = AutoModelForCausalLM.from_pretrained("bigcode/tiny_starcoder_py", use_cache=False)
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))
tokenizer.pad_token_id = tokenizer.eos_token_id
model.config.end_token_id = tokenizer.eos_token_id
model.config.pad_token_id = model.config.eos_token_id

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 49152. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


In [7]:
# Set up the datasets
data_path = "/kaggle/input/data-model-files/test_policy.parquet"
train_dataset = TLDRDataset(
    data_path,
    tokenizer,
    "train",
    max_length=256,
)

In [8]:
for i in train_dataset:
    print(i["input_ids"], i["labels"])
    break

tensor([ 7100,   613,  2918,   780,    44,   540,    33, 40186,   203, 13777,
           44,  3110,   428,    35,    43,   506,    79,   623,  1672, 11970,
          428,    35,    43,   488,   614,   646,  3654,   415,   439,  1631,
         1159, 16661,  1246,  6366,   973,  3425,    32,   203,  3705,    44,
        12000, 17964,  3638,  1548,    32,   439,  9845,   458,  7735,  1330,
         5133, 31695,   432,   312,  7000,   372,  7660,   544,  2442,    30,
         1273,   439,  4763,  2583, 42289,   312,  3493,   963,   432,  1672,
         7713,  1412,   561, 12767,   372,   458, 18734,   308,    59,  4763,
         5054,  1755,  1591, 12112,  2670,    30,   461,   436,  5075, 17510,
           30,   561,  1597,   963,   432,   322, 48385,   547,   203,   203,
         7558,   395,    19,  2770,    30,   312, 17142,   432, 22599, 14818,
           30,   439,  7307, 29220,   372,   458,  3932,   107,   544, 18660,
           30,  3919,   312,  9525,  2350,   688,   996,  4528, 

In [9]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())


True
2
0


In [10]:
# Prepare the trainer and start training
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=learning_rate,
    per_device_train_batch_size=train_batch_size,
#     per_device_eval_batch_size=eval_batch_size,
    fp16=False,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=2,
    warmup_steps=100,
    logging_steps=10,
)

In [11]:
training_args.device.index
# b63419aefda918e85878c10764b3510f470692d5

0

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
#     compute_metrics=compute_metrics,
#     data_collator=default_data_collator,
#     preprocess_logits_for_metrics=preprocess_logits_for_metrics
)
trainer.train()
# trainer.save_model(output_dir)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,10.188800
20,6.536500
30,2.513900
40,1.478500
50,1.186200
60,1.063200
70,1.016500
80,1.006000
90,0.993700
100,1.005500


TrainOutput(global_step=410, training_loss=1.3873600052624213, metrics={'train_runtime': 738.1286, 'train_samples_per_second': 17.756, 'train_steps_per_second': 0.555, 'total_flos': 2417790236491776.0, 'train_loss': 1.3873600052624213, 'epoch': 2.0})

In [13]:
trainer.save_model("summarization_policy_new/")

In [14]:
df.iloc[0]["label"]

'I really like this guy, but after having sex with him after only knowing him for a very brief period of time, I am worried that I may have ruined my chances of a relationship with him.'

In [15]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("summarization_policy_new/")
model_path = "bigcode/tiny_starcoder_py"

tokenizer = AutoTokenizer.from_pretrained(model_path, truncation=True, max_length=256, padding="max_length")
text = df.iloc[2]["prompt"]
tokenized_text = tokenizer(text, return_tensors="pt", max_length=256)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [16]:
tokenizer.decode(model.generate(**tokenized_text)[0])

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 203, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


"SUBREDDIT: r/relationships\nTITLE: The girl [26 F] I [22 M] have been seeing for a month didn't respond to me at all yesterday while hanging out with a friend [~30? M].\nPOST: She gets terrible service while at her house, but I texted her 3 times yesterday, 4-5 hours apart. She didn't call me until early this morning and left a voicemail that she was busy all day with a friend who showed up out of the blue.\n\nI saw that she posted a picture of the two of them out of her dead zone house on facebook before I texted her the last time.\n\nI don't mind that she hangs out with friends, and I know it's pretty early in the relationship, but am I wrong to be a little annoyed that she didn't respond until 24 hours after my first text?\nTL;DR: <|endoftext|>"

# Reward

In [17]:
import torch
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
from trl import RewardTrainer, SFTTrainer
from datasets import Dataset
import json
import pandas as pd
from transformers import Trainer, TrainingArguments


/opt/conda/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [18]:
##model path
MODEL_PATH = "bigcode/tiny_starcoder_py"
DATA_PATH = "/kaggle/input/data-model-files/test_reward.parquet"

In [19]:
df = pd.read_parquet(DATA_PATH)
df = df[:10]
raw_dataset = Dataset.from_pandas(df)
raw_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 10
})

In [20]:
##defininig the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)

In [21]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
def formatting_func(examples):
    kwargs = {"padding": "max_length",
              "truncation": True,
              "max_length": 256,
              "return_tensors": "pt"
              }

    # Prepend the prompt and a line break to the original_response and response-1 fields.
    prompt_plus_chosen_response = examples["prompt"] + "\n" + examples["chosen"]
    prompt_plus_rejected_response = examples["prompt"] + "\n" + examples["rejected"]

    # Then tokenize these modified fields.
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)

    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }

In [22]:
formatted_dataset = raw_dataset.map(formatting_func)
formatted_dataset = formatted_dataset.train_test_split()

  0%|          | 0/10 [00:00<?, ?ex/s]

In [23]:
model.config

GPTBigCodeConfig {
  "_name_or_path": "bigcode/tiny_starcoder_py",
  "activation_function": "gelu_pytorch_tanh",
  "architectures": [
    "GPTBigCodeForCausalLM"
  ],
  "attention_softmax_in_fp32": true,
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "inference_runner": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_batch_size": null,
  "max_sequence_length": null,
  "model_type": "gpt_bigcode",
  "multi_query": true,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": 3072,
  "n_layer": 20,
  "n_positions": 8192,
  "pad_key_length": true,
  "pre_allocate_kv_cache": false,
  "resid_pdrop": 0.1,
  "scale_attention_softmax_in_fp32": true,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.33.0",
  "use_cache": true,
  "validate_runner_input": tr

In [24]:
### Loading the TRL reward trainer and training the trainer
training_args = TrainingArguments(
        output_dir="rm_checkpoint/",
        num_train_epochs=1,
        logging_steps=10,
        gradient_accumulation_steps=1,
        save_strategy="steps",
        evaluation_strategy="steps",
        per_device_train_batch_size=2,
        per_device_eval_batch_size=1,
        eval_accumulation_steps=1,
        eval_steps=500,
        save_steps=500,
        warmup_steps=100,
        logging_dir="./logs",
        learning_rate=1e-5,
        save_total_limit=1,
        no_cuda=False
    )

In [25]:
trainer = RewardTrainer(model=model,
                        tokenizer=tokenizer,
                        train_dataset=formatted_dataset['train'],
                        eval_dataset=formatted_dataset['test'],
                        args= training_args
                        )
trainer.train()

/opt/conda/lib/python3.10/site-packages/trl/trainer/reward_trainer.py:105: FutureWarning: Using `transformers.TrainingArguments` for `args` is deprecated and will be removed in a future version. Please use `RewardConfig` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/reward_trainer.py:150: UserWarning: When using RewardDataCollatorWithPadding, you should set `max_length` in RewardConfig. It will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/reward_trainer.py:175: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your RewardConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` metho

Step,Training Loss,Validation Loss


TrainOutput(global_step=2, training_loss=0.7642489075660706, metrics={'train_runtime': 1.533, 'train_samples_per_second': 4.566, 'train_steps_per_second': 1.305, 'total_flos': 0.0, 'train_loss': 0.7642489075660706, 'epoch': 1.0})

In [26]:
trainer.save_model("rm_model/")

In [27]:
## inference the model
device = "cuda:0"
tokenizer = AutoTokenizer.from_pretrained("bigcode/tiny_starcoder_py")
rm_model = AutoModelForCausalLM.from_pretrained("rm_model/")
rm_model.to(device)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token


In [28]:
def get_score(model, tokenizer, prompt, response):
    device = "cuda:0"
    instructions = tokenizer.encode_plus(prompt,
                                       response,
                                       padding="max_length",
                                       max_length=256,
                                       return_tensors="pt",
                                        truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**instructions)

    logits = outputs[0]

    return logits


In [29]:
# usage with prompt
prompt = df.iloc[0]["prompt"]
example_prefered_response = df.iloc[0]["chosen"]
example_unprefered_response = df.iloc[0]["rejected"]

In [30]:
loss1 = get_score(model, tokenizer, prompt, example_prefered_response)
loss2= get_score(model, tokenizer, prompt, example_unprefered_response)

In [31]:
from torch import nn
loss = -nn.functional.logsigmoid(loss1 - loss2).mean()

In [32]:
tokenizer.decode(torch.max(loss1, axis=-1).indices[0])

'_DD__\n       "r/\n: Relationship RelationshipRelationship0]0ARCH\nlsriend\n2//M]\n [ [ [ [ his was to the gir\n a friends\n\n\n:\n [lfriend [ my have a aull with my19 minutes ago\n\n"""1 updated:** girlfriend was through my Facebook..** I my my Facebook..** my  of lf.\n\n** was ading for my new personirl** was was to getellpping my my my last** but I was a in\n\n** have ali ** of to,  tolirt me me girirl. and she had working about me.. the few of gir.1viously). was\'t find that was it\n\n** was it about  twoirl and the was the Facebook, the  and she birthdayin historyirl) was in March,\n to, f that were flited, I my g.. f-ing on her.\n girl wass; I1 [irllfriend, 19 months. to my Facebook.. my permission. I them messages. her.lirty with her coupleirl.\n found my with me after I through more with\n'

# ppo

In [33]:
import torch
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
from trl import RewardTrainer, SFTTrainer
from datasets import Dataset
import json
import pandas as pd
from transformers import Trainer, TrainingArguments
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model

In [34]:
from transformers import AutoTokenizer, pipeline
from datasets import Dataset
import torch


In [35]:
##model path
MODEL_PATH = "rm_model/"
DATA_PATH = "/kaggle/input/data-model-files/test_reward.parquet"

In [63]:
df = pd.read_parquet(DATA_PATH)
df = df[:1000]


In [64]:
df.to_csv('prompts_data.csv')

In [65]:
df

,prompt,chosen,rejected
0,SUBREDDIT: r/relationships\nTITLE: My [21/M] g...,TL;DR: My Girlfriend of 15 months went throug...,TL;DR: My girlfriend and I broke up after she...
1,SUBREDDIT: r/relationships\nTITLE: My [21/M] g...,TL;DR: My girlfriend found messages in my Fac...,TL;DR: My girlfriend and I broke up after she...
2,SUBREDDIT: r/relationships\nTITLE: My [21/M] g...,TL;DR: My girlfriend found messages in my Fac...,TL;DR: Girlfriend went through my Facebook ac...
3,SUBREDDIT: r/relationships\nTITLE: My [21/M] g...,TL;DR: My Girlfriend of 15 months went throug...,TL;DR: Girlfriend went through my Facebook ac...
4,SUBREDDIT: r/relationships\nTITLE: My [21/M] g...,TL;DR: My girlfriend and I broke up after she...,TL;DR: Girlfriend went through my Facebook ac...
...,...,...,...
995,SUBREDDIT: r/relationships\nTITLE: I [18 M] ju...,"TL;DR: Met a girl, went on first date and had...",TL;DR: I [18M] just got cancelled on for my s...
996,SUBREDDIT: r/relationships\nTITLE: I [18 M] ju...,"TL;DR: Met a girl, went on first date and had...",TL;DR: I [18M] just got cancelled on for my s...
997,SUBREDDIT: r/relationships\nTITLE: I [18 M] ju...,TL;DR: I [18M] just got cancelled on for my s...,TL;DR: I [18M] just got cancelled on for my s...
998,SUBREDDIT: r/relationships\nTITLE: I (26M) am ...,TL;DR: I (26M) am having a really hard time m...,TL;DR: I can't seem to move on from my first ...


In [59]:
!pip install datasets==2.11.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [62]:
!pip install pandas==1.5.3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [68]:
dataset = load_dataset('csv',data_files='/kaggle/working/prompts_data.csv')
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

TypeError: read_csv() got an unexpected keyword argument 'mangle_dupe_cols'

In [36]:
#df = pd.read_parquet(DATA_PATH)
#df = df[:1000]
#dataset = Dataset.from_pandas(df)
#dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 1000
})

In [54]:
df.dtypes


prompt      object
chosen      object
rejected    object
dtype: object

In [55]:
df['prompt'] = df['prompt'].astype('str')
df['chosen'] = df['chosen'].astype('str')
df['rejected'] = df['rejected'].astype('str')


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   prompt    1000 non-null   object
 1   chosen    1000 non-null   object
 2   rejected  1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


In [53]:
df

,prompt,chosen,rejected
0,SUBREDDIT: r/relationships\nTITLE: My [21/M] g...,TL;DR: My Girlfriend of 15 months went throug...,TL;DR: My girlfriend and I broke up after she...
1,SUBREDDIT: r/relationships\nTITLE: My [21/M] g...,TL;DR: My girlfriend found messages in my Fac...,TL;DR: My girlfriend and I broke up after she...
2,SUBREDDIT: r/relationships\nTITLE: My [21/M] g...,TL;DR: My girlfriend found messages in my Fac...,TL;DR: Girlfriend went through my Facebook ac...
3,SUBREDDIT: r/relationships\nTITLE: My [21/M] g...,TL;DR: My Girlfriend of 15 months went throug...,TL;DR: Girlfriend went through my Facebook ac...
4,SUBREDDIT: r/relationships\nTITLE: My [21/M] g...,TL;DR: My girlfriend and I broke up after she...,TL;DR: Girlfriend went through my Facebook ac...
...,...,...,...
995,SUBREDDIT: r/relationships\nTITLE: I [18 M] ju...,"TL;DR: Met a girl, went on first date and had...",TL;DR: I [18M] just got cancelled on for my s...
996,SUBREDDIT: r/relationships\nTITLE: I [18 M] ju...,"TL;DR: Met a girl, went on first date and had...",TL;DR: I [18M] just got cancelled on for my s...
997,SUBREDDIT: r/relationships\nTITLE: I [18 M] ju...,TL;DR: I [18M] just got cancelled on for my s...,TL;DR: I [18M] just got cancelled on for my s...
998,SUBREDDIT: r/relationships\nTITLE: I (26M) am ...,TL;DR: I (26M) am having a really hard time m...,TL;DR: I can't seem to move on from my first ...


In [39]:
sentiment_pipe_kwargs = {"top_k": None, "return_all_scores": True,"batch_size": 16,"function_to_apply": "none"}

config = PPOConfig(model_name=MODEL_PATH, steps=51200, learning_rate=1.41e-5)

txt_in_len = 5
txt_out_len = 20
seed = 1

In [40]:
dataset = dataset.rename_columns({"prompt": "review"})
dataset = dataset.filter(lambda x: len(x["review"]) > 500, batched=False)
dataset = dataset.map(lambda x: {"review": x["review"][:1000]}, batched=False)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [41]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

In [46]:
dataset

Dataset({
    features: ['review', 'chosen', 'rejected', 'input_ids', 'query'],
    num_rows: 1000
})

In [51]:
print(dataset)

Dataset({
    features: ['review', 'chosen', 'rejected', 'input_ids', 'query'],
    num_rows: 1000
})


In [42]:
txt_in_len = 5
txt_out_len = 32
seed = 1

dataset = dataset.map(
    lambda x: {"input_ids": tokenizer.encode(" " + x["chosen"], return_tensors="pt", truncation=True, padding="max_length", max_length=32)[0]},
    batched=False,
)
dataset = dataset.map(lambda x: {"query": tokenizer.decode(x["input_ids"])}, batched=False)
#dataset = dataset[:20480]

#dataset = Dataset.from_dict(dataset)
dataset.set_format("pytorch")

  0%|          | 0/1000 [00:00<?, ?ex/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [47]:
dataset.set_format(type='torch',columns=['review', 'chosen', 'rejected', 'input_ids', 'query'])

In [48]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32)


In [58]:
iter(dataloader)

In [57]:
dataloader['review']

TypeError: 'DataLoader' object is not subscriptable

In [50]:
next(iter(dataloader))

TypeError: new(): invalid data type 'numpy.str_'

In [72]:
type(dataset['chosen'])

TypeError: can't convert np.ndarray of type numpy.str_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [44]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [45]:
print(dataset)

Dataset({
    features: ['review', 'chosen', 'rejected', 'input_ids', 'query'],
    num_rows: 1000
})


In [76]:
#print(type(dataset['review']))
#print(type(dataset['chosen']))
print(type(dataset['rejected']))
print(type(dataset['input_ids']))
print(type(dataset['query']))



TypeError: can't convert np.ndarray of type numpy.str_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [46]:
rf_model_path = "rm_model/"
starcoder_model = AutoModelForCausalLMWithValueHead.from_pretrained(rf_model_path)
starcoder_model_ref = AutoModelForCausalLMWithValueHead.from_pretrained(rf_model_path)
starcoder_tokenizer = AutoTokenizer.from_pretrained(rf_model_path)

In [47]:
dataset

Dataset({
    features: ['review', 'chosen', 'rejected', 'input_ids', 'query'],
    num_rows: 1000
})

In [48]:
#optimizer = torch.optim.SGD(starcoder_model.parameters(), lr=config.learning_rate)
ppo_trainer = PPOTrainer(config, starcoder_model, starcoder_model_ref, starcoder_tokenizer, dataset=dataset, data_collator=collator)

In [51]:
ctrl_str = ["[negative]", "[positive]"]
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu" 
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # this should be handled by accelerate
ctrl_tokens = dict((s, starcoder_tokenizer.encode(s, return_tensors="pt").squeeze().to(device)) for s in ctrl_str)


In [52]:
def pos_logit_to_reward(logit, task):
    """
    Take the positive sentiment logit and scale it for the task.
        task [negative]: reward = -logit
        task [neutral]: reward = -2*abs(logit)+4
        task [positive]: reward = logit
    """
    for i in range(len(logit)):
        if task[i] == "[negative]":
            logit[i] = -logit[i]
        elif task[i] == "[positive]":
            pass
        else:
            raise ValueError("task has to be in [0, 1, 2]!")
    return logit

In [53]:
pos_logit_to_reward(torch.Tensor([4, 4]), ctrl_str)

tensor([-4.,  4.])

In [54]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": starcoder_tokenizer.eos_token_id,
    "max_new_tokens": 32,
    "eos_token_id": -1,
}

In [55]:
def get_score(model, tokenizer, responses):
    positive_logist = []
    for i in responses:
        instructions = tokenizer.encode_plus(
                                           i,
                                           padding="max_length",
                                           max_length=32,
                                           return_tensors="pt")
        with torch.no_grad():
            outputs = model(**instructions)

        logits = outputs[0].mean()
        positive_logist.append(logits)

    return positive_logist


In [57]:
from random import choices
from tqdm import tqdm
import time
import numpy as np
tqdm.pandas()

In [60]:
type(ppo_trainer)

trl.trainer.ppo_trainer.PPOTrainer

In [61]:
type(ppo_trainer.dataloader)

accelerate.data_loader.DataLoaderShard

In [69]:
for epoch in range(1):
    print(epoch)
    for batch in tqdm(ppo_trainer.dataloader):
        print(batch)

0


  0%|          | 0/3 [00:00<?, ?it/s]


TypeError: new(): invalid data type 'numpy.str_'

In [67]:
for epoch,batch in tqdm(enumerate(ppo_trainer.dataloader)):
    print(type(epoch))
    print(type(batch))

0it [00:00, ?it/s]


TypeError: new(): invalid data type 'numpy.str_'

In [66]:
print(ppo_trainer.dataloader)

In [ ]:
for epoch,batch in tqdm(enumerate(ppo_trainer.dataloader)):
    print(epoch)
    print(batch)